[Novo Coronavírus (2019-nCov) - Situação Epidemiológica no ESP](http://www.saude.sp.gov.br/cve-centro-de-vigilancia-epidemiologica-prof.-alexandre-vranjac/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus-covid-19/situacao-epidemiologica)

In [4]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from folium import plugins
import tabula
import pandas as pd
import os
import re

In [5]:
tabula.__version__

'2.1.0'

In [2]:
M = re.compile(r'([A-ZùÙÁÁÀÂÄÃÉÈÊÍÌÓÒÔÖÕÚÙÜÇ ]{1,100})([0-9- ]{1,10})')

In [20]:
df = tabula.read_pdf("../bulletins/boletim_51_17-04-20.pdf", pages='all')

Got stderr: abr 19, 2020 10:29:47 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:29:47 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:29:47 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:29:47 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:29:49 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:29:49 AM org.apache.pdfbox.pdmode

In [25]:
df[1].columns

Index(['Casos de COVID-19 por município de residência, São Paulo, 21 de janeiro a 17 de abril de 2020'], dtype='object')

In [28]:
def parse_ugly_df(ugly_df):
    data = []
    for _, row in ugly_df.iterrows():
        string = ''
        for col in row.index:
            string  += " " + str(row[col])
        data.append(string)
    
    del data[0]
    data2 = []
    for string in data:
        data2 += M.findall(string)
    data3 = []
    for item in data2:
        cases = item[1]
        cases = cases.split(" ")
        cases = [int(s) if s != '-' else 0 for s in cases if s != '']
        data3.append([item[0][:-1]] + cases)
    petty_df = pd.DataFrame(data=data3, columns=["municipio", "confirmados", "obitos"])
    return petty_df

In [5]:
arr = os.listdir("../bulletins")
arr

['boletim_51_17-04-20.pdf',
 'boletim_47_13-04-20.pdf',
 'boletim_48_14-04-20.pdf',
 'boletim_36_02-04-20.pdf',
 'boletim_46_12-04-20.pdf',
 'boletim_42_08-04-20.pdf',
 'boletim_50_16-04-20.pdf',
 'boletim_40_06-04-20.pdf',
 'boletim_43_09-04-20.pdf',
 'boletim_37_03-04-20.pdf',
 'boletim_33_30-03-20.pdf',
 'boletim_34_31-03-20.pdf',
 'boletim_41_07-04-20.pdf',
 'boletim_45_11-04-20.pdf',
 'boletim_30_25-03-20.pdf',
 'boletim_52_18-04-20.pdf',
 'boletim_35_01-04-20.pdf',
 'boletim_44_10-04-20.pdf',
 'boletim_38_04-04-20.pdf',
 'boletim_32_27-03-20.pdf',
 'boletim_31_26-03-20.pdf',
 'boletim_39_05-04-20.pdf',
 'boletim_49_15-04-20.pdf']

In [29]:
def rightDf(df):
    right_column = "Casos de COVID-19 por município de residência, São Paulo"
    columns = df.columns
    for col in columns:
        if right_column in col:
            return True
    return False
    


df_list = []
for pdf in arr:
    boletim_and_date = pdf.split("_")
    boletim = boletim_and_date[1]
    date = boletim_and_date[2][:-4] + "20"
    file_path = "../bulletins/" + pdf
    ugly_df_list = tabula.read_pdf(file_path, pages='all')
    for ugly_df in ugly_df_list:
        if ugly_df.shape[0] > 0:
            rigth_df = rightDf(ugly_df)
            if rigth_df:
                break
    petty_df = parse_ugly_df(ugly_df)
    petty_df["data"] = date
    petty_df["boletim"] = boletim
    df_list.append(petty_df)
    
df_covid = pd.concat(df_list)

Got stderr: abr 19, 2020 10:42:45 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:42:45 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:42:46 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:42:46 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:42:47 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:42:47 AM org.apache.pdfbox.pdmode

Got stderr: abr 19, 2020 10:43:13 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:13 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:13 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Bold are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:14 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:14 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:14 AM org.apache.pdfbox.p

Got stderr: abr 19, 2020 10:43:47 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:48 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:48 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:48 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:49 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri,Italic are not implemented in PDFBox and will be ignored
abr 19, 2020 10:43:49 AM org.apache.pdfbox

In [30]:
df_covid

,municipio,confirmados,obitos,data,boletim
0,ADAMANTINA,1.0,0.0,17-04-2020,51
1,GUARUJÁ,27.0,2.0,17-04-2020,51
2,PILAR DO SUL,3.0,0.0,17-04-2020,51
3,ÁGUAS DE LINDÓIA,1.0,1.0,17-04-2020,51
4,GUARULHOS,295.0,28.0,17-04-2020,51
...,...,...,...,...,...
200,IGNORADO,9.0,0.0,15-04-2020,49
201,GUARATINGUETA,1.0,0.0,15-04-2020,49
202,PEDRA BELA,1.0,0.0,15-04-2020,49
203,GUARUJA,17.0,0.0,15-04-2020,49


In [169]:
df_covid = df_covid.fillna(0)

In [176]:
df_covid = df_covid.sort_values(by=['boletim'])

In [181]:
df_covid = df_covid[["date", "boletim", "municipio", "confirmados", "obitos"]]

In [184]:
df_covid.columns = ["Dia", "Boletim", "Municipio", "Confirmados", "Obitos"]

In [185]:
df_covid

,Dia,Boletim,Municipio,Confirmados,Obitos
38,25-03-2020,30,SAO BERNARDO DO CAMPO,13.0,0.0
23,25-03-2020,30,TABOAO DA SERRA,1.0,1.0
22,25-03-2020,30,RIBEIRAO PIRES,1.0,0.0
20,25-03-2020,30,EMBU DAS ARTES,1.0,0.0
19,25-03-2020,30,SUZANO,1.0,0.0
...,...,...,...,...,...
105,11-04-2020,45,SÃO SEBASTIÃO,4.0,1.0
104,11-04-2020,45,MAIRIPORÃ,7.0,2.0
103,11-04-2020,45,CARAGUATATUBA,2.0,1.0
111,11-04-2020,45,SERTÃOZINHO,2.0,0.0


In [186]:
df_covid.to_csv("data/covid-19-municipios-sp.csv", index=False)

In [209]:
df_covid2 = pd.read_csv("data/covid-19-municipios-sp.csv")

In [210]:
sao_paulo = df_covid2[df_covid2["Municipio"] == "SÃO PAULO"]
sao_paulo

,Dia,Boletim,Municipio,Confirmados,Obitos
58,26-03-2020,31,SÃO PAULO,899.0,53.0
80,27-03-2020,32,SÃO PAULO,1044.0,62.0
147,30-03-2020,33,SÃO PAULO,1233.0,103.0
225,31-03-2020,34,SÃO PAULO,1885.0,121.0
304,01-04-2020,35,SÃO PAULO,2418.0,144.0
383,02-04-2020,36,SÃO PAULO,2815.0,164.0
416,03-04-2020,37,SÃO PAULO,3202.0,186.0
505,04-04-2020,38,SÃO PAULO,3496.0,212.0
707,06-04-2020,40,SÃO PAULO,3754.0,244.0
894,07-04-2020,41,SÃO PAULO,4258.0,296.0


In [211]:
sao_paulo.Obitos.diff()

58       NaN
80       9.0
147     41.0
225     18.0
304     23.0
383     20.0
416     22.0
505     26.0
707     32.0
894     52.0
1044    43.0
1106    45.0
1336    25.0
1495    13.0
Name: Obitos, dtype: float64

In [201]:
1044 - 899

145

# Obtendo qt casos

In [215]:
df_list = []
for municipio in df_covid2.Municipio.unique():
    d = df_covid2[df_covid2["Municipio"] == municipio]
    d["Qt_Novos_Casos"] = d.Confirmados.diff()
    d["Qt_Novas_Mortes"] = d.Obitos.diff()
    df_list.append(d)
    
df_covid3 = pd.concat(df_list)

/home/cloves/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/cloves/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [216]:
df_covid3 = df_covid3.sort_values(by=['Boletim', "Municipio"])

# Obtendo latitude e longitude

In [220]:
def do_geocode(address):
    geolocator = Nominatim(user_agent="clovesgtx@hotmail.com")
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [223]:
municipios = [municipio + " - SP" for municipio in df_covid2.Municipio.unique()]
municipios[:5]

['SAO BERNARDO DO CAMPO - SP',
 'TABOAO DA SERRA - SP',
 'RIBEIRAO PIRES - SP',
 ' EMBU DAS ARTES - SP',
 'SUZANO - SP']

In [ ]:
latitude = []
longitude = []
for m in municipios:
    location = do_geocode(endereco)
    latitude = location.latitude
    longitude = location.longitude